# VOSviewer Visualization

예시 데이터를 통해 VOSviewer에서 사용하고 있는 mapping 방법을 구현해 보자. 

REF
* [VOS: A New Method for Visualizing Similarities
between Objects](https://repub.eur.nl/pub/7654/ERS%202006%20020%20LIS.pdf)

## Overview




### Data


n개의 node 각각이 고차원(R차원) 평면에 위치해 있다고 생각해 보자.  

이들 벡터를 row vector로 하는 matrix $E = ( e_{k} )$를 상정할 수 있다. `E.shape==(n,R)`

node들이 맺고 있는 관계를 통해 n × n 차원의 co-occurrence Matrix $C = ( c_{ij} )$를 구할 수 있다.  `C.shape==(n,n)`

* $c_{ij}$ : the number of links (e.g., co-occurrence links,
co-citation links, or bibliographic coupling links) between nodes i and j. 
* 단, $c_{ij} = c_{ji} ≥ 0$, $c_{ii} = 0$으로 한다.


$C$로 부터 각 노드 사이의 유사성을 의미하는 similarity matrix $S = (s_{ij})$를 구할 수 있다. VOSviewer에서는 Association strength를 적용시키고 있다. 구하는 방법은 아래와 같다. 

$$
s_{ij} = { { c_{ij}  \over m } \over { { c_i \over m } \cdot {c_j \over m} }  }    = { mc_{ij} \over c_ic_j }
$$

$$
s_i = \sum_{j \neq i} c_{ij}   \mbox{   and   }  m = \sum_i c_i
$$


### Visualization

노드들의 관계를 2차원 혹은 3차원에 표현하기 위해서는 차원축소 기법을 통해 $E$를 `(n,2)` 혹은 `(n,3)` 차원으로 축소하여 타나낼 수 있다. 

그러나 Network 데이터에서는 $E$가 알려져 있지 않고 node들 상의 관계인 $C$만 주어지는 경우가 많다. 따라서 $C$나 $S$를 통해 node의 상대적인 거리를 나타내야 한다. 

시각화를 위해 p차원(p는 주로 2가 됨)으로 축소된 좌표를 row vector로 하는 Matrix $X=(x_k)$를 상정해 보자. `X.shape == (n, p)` 


$$
d_{ij} = ||x_i-x_j|| = \sqrt[\leftroot{-2}\uproot{2}]{\sum_{k=1}^p (x_{ik} - x_{jk})^2}
$$


위와 같이 각 node 들의 공간 상의 거리를 표현했을 때, 

$$
V(x_1, ...,x_n) = \sum_{i<j} {s_{ij} d^2_{ij}}
$$

위의 수식을 최소화 하는 $X$를 구하는 것을 목적으로 한다. 

단, 한 점으로 수렴되는 것을 막기 위해 아래와 같은 조건을 둔다. 

 $$
 \sum_{i<j} d_{ij} = 1
$$

정리하면, $X$를 구하는 것은 $V$를 cost function으로 하는 최적화 문제가 된다. 


## Sample Example


Sample data를 가지고 VOSviewer의 방법을 통해 시각화를 결과를 도출해 보자. 

Sample data로는, 아래와 같은 조건에 맞는 51 × 51 similarity matrix $S = (s_{ij})$를 구현하였다. 예시 데이터는 예시는 "[VOS: A New Method for Visualizing Similarities
between Objects](https://repub.eur.nl/pub/7654/ERS%202006%20020%20LIS.pdf)"에서 소개된 내용을 사용하였다. 

$$
S=
\begin{cases}
8,  & \mbox{if } 1 \le | i - j | \le 3 \\
7,  & \mbox{if } 4 \le | i - j | \le 6 \\
... \\
1,  & \mbox{if } 22 \le | i - j | \le 24 \\
0,  & \mbox{otherwise } 
\end{cases}
$$

인용문헌에서와 같이 여기에서도 MDS(multidimensional scaling)와 결과를 비교할 예정이다. MDS는 similarity가 아니라 distance를 입력값으로 하기 때문에 아래와 같이 $D$를 구하였다. 

$$
S=
\begin{cases}
1,  & \mbox{if } 1 \le | i - j | \le 3 \\
2,  & \mbox{if } 4 \le | i - j | \le 6 \\
... \\
8,  & \mbox{if } 22 \le | i - j | \le 24 \\
0,  & \mbox{otherwise } 
\end{cases}
$$

주의 해야할 점은, $S$든 $D$든 $i=j$일 경우에는 값을 $0$으로 해주어야 한다는 점이다. $D$의 경우에는 자기 자신과의 거리이기 때문에 0이 당연하게 느껴지지만, $S$에서도 추후 연산에서 $i=j$의 값들은 사용하지 않기 때문에 연산의 오류를 제거하기 위해 0으로 해 둔다. 

### 데이터 구현

In [1]:
import numpy as np

In [2]:
n = 51
sample_sim = np.zeros( (n, n) )
sample_dis = np.zeros( (n, n) )
lim = 8

for i in range(0, n):
  for j in range(0, n):
    d = np.abs( i - j )
    if d == 0 : continue
    if d > 24 : continue
    k_ = (d-1) // 3
    sample_sim[i][j] = 8 - k_
    sample_dis[i][j] = k_

print( sample_sim )
print( sample_dis )

[[0. 8. 8. ... 0. 0. 0.]
 [8. 0. 8. ... 0. 0. 0.]
 [8. 8. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 8. 8.]
 [0. 0. 0. ... 8. 0. 8.]
 [0. 0. 0. ... 8. 8. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Visualization with MDS

In [3]:
from sklearn.manifold import MDS, smacof
from sklearn.metrics import pairwise_distances


In [4]:
# SMACOF
## https://scikit-learn.org/stable/modules/generated/sklearn.manifold.smacof.html
sample_sim_mds, _ = smacof( sample_sim, n_components=2 )
sample_dis_mds, _ = smacof( sample_dis, n_components=2 )

In [5]:
from bokeh.plotting import figure, show, output_notebook

def scatter_plot( embedded, title ):
    lst = embedded.tolist()
    x, y = zip( *lst )

    # add a circle renderer with a size, color, and alpha
    p = figure( plot_width=400, plot_height=400, title=title )
    p.scatter( x, y, size=2, color="navy", alpha=0.5)
    return p


In [6]:
# show the results
output_notebook() 
# show( scatter_plot( sample_sim_mds, "SIM MDS" ) )
show( scatter_plot( sample_dis_mds, "DIS MDS" ) )

Loading BokehJS ...

### Visualization with VOSviewer methon 

최적화 문제이므로 Tensorflow를 사용하였다. 

In [7]:
import tensorflow as tf
tf.set_random_seed( 99 )  # for reproducibility

In [8]:
n, _ = sample_sim.shape
sum_constrain = n * (n-1) / 2

# placeholders for a tensor that will be always fed.
S = tf.placeholder( tf.float32, [n, n] )

# embedded
X = tf.Variable( tf.random_normal( [n, 2] ), name='embedded')

# Pairwise Euclidean distance ( Squared )
"""
# https://stackoverflow.com/questions/37009647/compute-pairwise-distance-in-a-batch-without-replicating-tensor-in-tensorflow
"""
r_ = tf.reduce_sum( X * X, 1 )
# turn r into column vector
r = tf.reshape( r_, [-1, 1] )
d_square = r - 2 * tf.matmul( X, tf.transpose( X ) ) + tf.transpose( r )

In [9]:
cost = tf.reduce_sum( S * d_square ) 

# Apply Constrain1 ... fail
# zr = tf.zeros_like(S)
# proximate = tf.reduce_sum( tf.maximum( d_square , zr ) )
# c1 = tf.reduce_sum( tf.multiply( S, proximate ) )
# c2 = tf.reduce_sum( tf.sqrt( proximate ) ) 
# cost =  c1 - c2

# Apply Constrain2 ... fail
# alpha = 0.1
# balance = tf.reduce_sum( tf.sqrt( d ) )
# update1 = X.assign( X * ( 1 - alpha ) )
# update2 = X.assign( X * ( 1 + alpha ) )

In [10]:
# Try 01
learning_rate = 1e-5
train = tf.train.GradientDescentOptimizer( learning_rate=learning_rate ).minimize( cost )

# Try 02
# learning_rate = 0.00001
# optimizer_name = ["ProximalGradientDescentOptimizer", "FtrlOptimizer", "AdamOptimizer", "RMSPropOptimizer"]
# train = getattr( tf.train, optimizer_name[0])( learning_rate=learning_rate ).minimize( cost )

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run( tf.global_variables_initializer() )

for step in range(4001):
    _, cost_val, _ = sess.run([ train, cost, X ], feed_dict={ S: sample_sim } )
#     Apply Constrain
#     _, cost_val, _, balance_val = sess.run([ train, cost, X, balance ], feed_dict={ S: sample_sim } )
#     if balance_val < sum_constrain: sess.run( update1, feed_dict={ S: sample_sim } )
#     if balance_val > sum_constrain: sess.run( update2, feed_dict={ S: sample_sim } )
    if step % 400 == 0:
        print(step, "Cost: ", cost_val )

embedded = sess.run( X, feed_dict={ S: sample_sim } )


0 Cost:  38191.36
400 Cost:  335.42523
800 Cost:  43.573425
1200 Cost:  11.309889
1600 Cost:  3.0924783
2000 Cost:  0.8488287
2400 Cost:  0.23306459
2800 Cost:  0.063992485
3200 Cost:  0.017568814
3600 Cost:  0.004830517
4000 Cost:  0.0013193265


In [11]:
output_notebook() 
show( scatter_plot( embedded, "VOSviewer" ) )

Loading BokehJS ...

## Conclusion 

지금까지 VOSviewer에서 설명한 시각화 방법을 이해하고 구현해 보기 위하여 [VOS: A New Method for Visualizing Similarities
between Objects](https://repub.eur.nl/pub/7654/ERS%202006%20020%20LIS.pdf)에 언급된 예시를 python 코드로 만들어 보았다. 

시행착오가 많았지만, 참고문헌과 동일한 결과를 얻을 수 있었다. 

MDS로 구현한 plot과 VOSviewer 방식으로 구현한 plot의 결과 서로 다르게 나타났다. 

***


다만, 수렴을 방지하기 위한 조건인 $\sum_{i<j} d_{ij} = 1$을 코드로 어떻게 구현해야 할지 몰라 생략하였는데, 다행히도 주어진 예시에 대해서는 예상했던 결과를 얻을 수 있었다. 

참고로 [A unified approach to mapping and clustering of bibliometric networks]()에서는 수렴을 막기 위한 조건을 따로 주지 않고 cost function을 아래와 같이 수정해 표현하였다.  

$$
V(x_1, ...,x_n) = \sum_{i<j} {s_{ij} d^2_{ij}}  -  \sum_{i<j} d_{ij}
$$

하지만 위의 공식을 적용하여 tensorflow에 적용하였더니 $\sum_{i<j} d_{ij}$ 부분이 크게 발산하면서 최적화가 이루어지지 않았다. cost function이 convex한 형태를 띄지 않는 모양이다. 


